In [100]:
from math import ceil
import time, re, ast, sys, urllib, random, string, requests
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from selenium.webdriver.firefox.webdriver import FirefoxProfile
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from itertools import cycle
from header_list import user_agent_list
from proxies_list import proxies_list_
from INITIAL_SCRAPPER_FUNCTIONS import *
from PARSER_FUNCTIONS import *
from LIST_DF_FUNCTIONS import *
from SEARCH_URL_GEN import *
from GET_SEARCH_URL import *
from GET_RESULTS import *
ua = user_agent_list
proxies = proxies_list_

In [101]:
proxies = ['12.183.155.91:39828', '12.2.202.242:8080', '50.224.173.189:8080', '12.175.211.121:59669',
           '71.43.26.18:46024', '72.35.40.34:8080', '50.224.173.190:8080', '24.227.222.4:53281',
           '64.58.192.36:41258', '72.4.34.79:53281', '172.83.138.45:53281', '72.95.19.165:57701',
           '65.173.110.234:8888', '162.236.204.234:60445', '47.210.133.169:43305', '64.214.77.66:8080',
           '209.97.233.3:61268', '73.0.14.154:8080', '76.72.92.108:35247', '75.102.135.249:40326',
           '12.189.124.100:54773', '64.49.66.69:51633', '50.224.173.179:8080', '98.198.60.95:54148',
           '52.144.107.142:60263', '75.142.138.44:42165', '204.249.203.201:50032', '38.134.10.106:53281',
           '173.162.101.209:32231', '50.252.132.229:31116', '12.218.209.130:53281', '65.49.167.6:8888',
           '172.73.162.185:56206', '74.0.244.146:52807', '67.198.99.138:60810', '76.10.248.156:8080',
           '45.30.170.25:8888', '24.119.8.100:53281', '108.170.48.187:3128', '140.227.60.114:3128',
           '64.192.234.98:53281', '72.43.146.42:59313', '74.143.136.6:8888', '142.147.112.246:59640',
           '71.183.100.76:42413', '166.130.163.233:34739', '63.246.49.113:8080', '64.19.116.82:60814',
           '66.232.186.7:53281', '12.190.124.122:8888', '50.236.148.254:39970', '96.57.221.236:42288',
           '66.91.106.224:39395', '66.153.222.162:38692', '50.235.111.161:45126', '74.84.121.206:55219',
           '74.122.78.18:44125', '24.60.229.154:35700', '199.231.171.106:53281', '74.132.148.53:53004',
           '209.201.21.140:49358', '96.3.212.186:38901', '24.111.55.159:59774', '31.207.59.64:51378',
           '70.167.232.47:35380', '74.117.26.133:54058', '23.224.151.232:808', '208.255.136.24:9999',
           '75.145.243.13:8888', '12.189.124.227:49361', '162.250.159.182:33877', '38.121.121.241:39391',
           '52.124.6.226:44894']

In [102]:
def proxie_check(proxies):
    default_list = []
    url = 'https://httpbin.org/ip'
    for i in range(0, len(proxies)):
        proxy = proxies[i]
        print(i+1)
        start_time = time.time()
        try:
            response = requests.get(
                url, proxies={"http": proxy, "https": proxy})
            print(response.json())
            print(time.time() - start_time)
        except:
            print("Skipping. Connnection error")
            default_list.append(i+1)
            print(time.time() - start_time)
        print(default_list)
    return default_list
#proxie_check(proxies)

In [103]:
len(proxies)

73

In [22]:
cities_url = pd.read_csv('../Data/zip_url_search_list.csv', header=None)
city_url_list = cities_url[0].tolist()

In [23]:
prop_count_per_city_list = []

In [34]:
def session_creator(proxy, ua, url):
    header = random.sample(ua, 1)[0]
    session = requests.Session()
    session.proxies = {"http": proxy, "https": proxy}
    req = session.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

In [35]:
i = 0

In [36]:
city_url_list = random.sample(city_url_list, len(city_url_list))
def areas_extractor_url(proxies, prp_list, url_list, ua):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    for url in url_list:
        try:
            start_time = time.time()
            soup = session_creator(proxy, ua, url)
            time.sleep(random.uniform(1.5, 3.8))
            all_count = soup.find('meta', {'name': 'description'})
            print(all_count.text)
            print(time.time() - start_time)
            if len(all_count) > 0:
                prp_list.append(all_count)
                url_list.remove(url)
                print(f'url left: {len(url_list)}')
                print(f'url done: {len(prp_list)}')
            else:
                print('captcha')
        except:
            print("Connnection error")
            proxies.remove(proxy)
            print(len(proxies))
            return prp_list, url_list
    return prp_list, url_list


while i < len(proxies):
    prop_count_per_city_list, city_url_list = areas_extractor_url(
        proxies, prop_count_per_city_list, city_url_list, ua)
    i+=1

12
37.61.224.231:8195
[<meta content="Zillow has 6,644 homes for sale in Jacksonville FL. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
6.419410943984985
url left: 199
url done: 111
[<meta content="Zillow has 382 homes for sale in Hattiesburg MS. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.877525806427002
url left: 198
url done: 112
[<meta content="Zillow has 356 homes for sale in Flushing New York. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
7.0229010581970215
url left: 197
url done: 113
[<meta content="Zillow has 562 homes for sale in Charleston WV. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
6.213216543197632
url left: 196
url do

[<meta content="Zillow has 620 homes for sale in Bismarck ND. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
5.265373468399048
url left: 166
url done: 144
[<meta content="Zillow has 257 homes for sale in Bethlehem PA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
3.3416354656219482
url left: 165
url done: 145
[<meta content="Zillow has 697 homes for sale in Alexandria VA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.461730003356934
url left: 164
url done: 146
[<meta content="Zillow has 877 homes for sale in Modesto CA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.025703191757202
url left: 163
url done: 147
[<meta content="Zillow has 3,

[<meta content="Zillow has 3,567 homes for sale in El Paso TX. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
3.5640177726745605
url left: 132
url done: 178
[<meta content="Zillow has 323 homes for sale in Lafayette IN. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
3.4986212253570557
url left: 131
url done: 179
[<meta content="Zillow has 1,281 homes for sale in Springfield MO. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.720622539520264
url left: 130
url done: 180
[<meta content="Zillow has 654 homes for sale in Garland TX. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
5.23098349571228
url left: 129
url done: 181
[<meta content="Zillow ha

[<meta content="Zillow has 3,418 homes for sale in Tucson AZ. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
3.8865966796875
url left: 98
url done: 212
[<meta content="Zillow has 1,308 homes for sale in Akron OH. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.207335948944092
url left: 97
url done: 213
[<meta content="Zillow has 412 homes for sale in Youngstown OH. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
5.1730873584747314
url left: 96
url done: 214
[<meta content="Zillow has 259 homes for sale in Waukegan IL. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.940746545791626
url left: 95
url done: 215
[<meta content="Zillow has 2,238 hom

[<meta content="Zillow has 1,239 homes for sale in Huntsville AL. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
20.256639003753662
url left: 64
url done: 246
[<meta content="Zillow has 946 homes for sale in Tacoma WA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
11.16304349899292
url left: 63
url done: 247
[<meta content="Zillow has 343 homes for sale in Trenton NJ. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
20.23131513595581
url left: 62
url done: 248
[<meta content="Zillow has 164 homes for sale in Berkeley CA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.848527193069458
url left: 61
url done: 249
[<meta content="Zillow has 163 hom

[<meta content="Zillow has 552 homes for sale in Boulder CO. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.839339733123779
url left: 30
url done: 280
[<meta content="Zillow has 153 homes for sale in Cambridge MA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.718138933181763
url left: 29
url done: 281
[<meta content="Zillow has 825 homes for sale in Vancouver WA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
3.7563509941101074
url left: 28
url done: 282
[<meta content="Zillow has 652 homes for sale in Tyler TX. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]
4.92648720741272
url left: 27
url done: 283
18
76.72.92.108:35247
[<meta content="

In [37]:
len(proxies)

27

In [88]:
total_act_pend_per_city = []
for num in prop_count_per_city_list:
    for i in num:
        total_act_pend_per_city.extend((re.findall(r'\d+', i['content'].replace(',',''))))
total_act_pend_per_city = [int(num) for num in total_act_pend_per_city]

In [89]:
city_name = []
for num in prop_count_per_city_list:
    for i in num:
        city_name.extend((re.findall(r'in\s(.+?)\.', i['content'])))
city_name = [num.replace(' New York', ' NY').replace(' Los Angeles', ' CA') for num in city_name]

In [97]:
act_pend_city_df = pd.DataFrame(columns=['city', 'tot_act_pend'])
act_pend_city_df['city'] = city_name
act_pend_city_df['tot_act_pend'] = total_act_pend_per_city
act_pend_city_df['num_pgs'] = ((act_pend_city_df['tot_act_pend']//25)+1)
act_pend_city_df.to_csv('../Data/act_pend_by_city.csv')

In [38]:
def strip_count(lst):
    lst = [str(num) for num in lst]
    lst = [num.replace('[<meta content="Zillow has ', '') for num in lst]
    lst = ap_prop_list = [num.replace(
        '. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/>]', '') for num in lst]
    lst = list(set(lst))
    return lst

new_prop_count_per_city_list = strip_count(prop_count_per_city_list)

In [40]:
new_prop_count_per_city_list

['346 homes for sale in Hayward CA',
 '1,500 homes for sale in Myrtle Beach SC',
 '2,247 homes for sale in McKinney TX',
 '562 homes for sale in Charleston WV',
 '946 homes for sale in Tacoma WA',
 '1,092 homes for sale in Buffalo NY',
 '1,397 homes for sale in Savannah GA',
 '889 homes for sale in Boise ID',
 '652 homes for sale in Augusta GA',
 '4,032 homes for sale in Atlanta GA',
 '607 homes for sale in Worcester MA',
 '2,017 homes for sale in Henderson NV',
 '1,014 homes for sale in Chandler AZ',
 '292 homes for sale in Kalamazoo MI',
 '1,991 homes for sale in Baton Rouge LA',
 '163 homes for sale in Corona New York',
 '620 homes for sale in Elgin IL',
 '2,205 homes for sale in Staten Island NY',
 '963 homes for sale in Fort Wayne IN',
 '594 homes for sale in Wilmington DE',
 '211 homes for sale in York PA',
 '826 homes for sale in Columbia MO',
 '197 homes for sale in Muskegon MI',
 '1,060 homes for sale in Des Moines IA',
 '1,744 homes for sale in Wichita KS',
 '1,807 homes for 

In [39]:
def create_df_num_homes(prp_list):
    re_all_num = r"\d+\s"
    re_all_zip = r"\s\d+"

    num_homes = []
    zip_homes = []

    for num in prp_list:
        num_homes.append(re.findall(re_all_num, num))
        zip_homes.append(re.findall(re_all_zip, num))

    num_homes = [[int(y) for y in x] for x in num_homes]
    zip_homes = [[int(y) for y in x] for x in zip_homes]

    num_all_homes = []
    zip_all_homes = []

    i = 0
    while i < len(prp_list):
        num_all_homes.extend(num_homes[i])
        zip_all_homes.extend(zip_homes[i])
        i += 1

    df = pd.DataFrame({"all_homes": num_all_homes}, index=zip_all_homes)
    all_pages = [ceil(num/25) for num in df['all_homes']]
    df['all_pages'] = all_pages
    return df


df_all = create_df_num_homes(new_prop_count_per_city_list)

ValueError: Shape of passed values is (1, 310), indices imply (1, 0)

In [ ]:
df_all

In [28]:
def zip_prop_count(zip_list, proxies, prp_list, ua, ezl):
    proxy = random.sample(proxies, 1)[0]
    print(proxies.index(proxy))
    print(proxy)
    for num in zip_list:
        url = 'https://www.redfin.com/zipcode/' + \
            str(num) + '/filter/property-type=house+condo+townhouse,' + \
            'include=sold-1yr,min-price=100k,min-baths=1,include=sold-1yr'
        try:
            start_time = time.time()
            soup = session_creator(proxy, ua, url)
            print(num)
            print(len(zip_list))
            all_count = soup.findAll('div', {'class': 'homes summary'})
            if len(str(all_count)) >= 20:
                print(all_count)
                print(time.time() - start_time)
                ezl.append(num)
                prp_list.append(all_count)
                zip_list.remove(num)
                print(len(zip_list) + len(prp_list))
            else:
                print("Captcha!!!!!")
        except:
            print("Skipping. Connnection error")
            proxies.remove(proxy)
            print(len(proxies))
            return prp_list, zip_list, proxies, ezl
    return prp_list, zip_list, proxies, ezl

In [ ]:
def each_page(proxy, ua, url):
    soup = session_creator(proxy, ua, url)
    # start_time = time.time()
    time.sleep(random.uniform(0, 1) * 3)
    # print(time.time() - start_time)
    full_soup = soup.findAll('a', {'class': 'bottom link-override'})
    full_address = [fas['title'] for fas in full_soup]
    home_link = ['https://www.redfin.com' +
                 str(hls.get('href')) for hls in full_soup]
    df = {'full_address': full_address, 'home_link': home_link}
    return df

In [ ]:
def links_for_props(proxies, url_list, main_df, ua):
    proxy = random.sample(proxies, 1)[0]
    print(f'proxy number: {proxy}')
    i = randint(0, (len(url_list) // 2))
    print(f'starting from url number: {i}')
    while i < len(url_list):
        url = url_list[i]
        try:
            b = random.uniform(0.75, 2.25)
            time.sleep(b)
            # start_time = time.time()
            # print(f'total left: {len(url_list)}')

            data = each_page(proxy, ua, url)
            df = pd.DataFrame(data)
            eds = {'full_address': [], 'home_link': []}
            if data['full_address'] != eds['full_address']:
                main_df = pd.concat([main_df, df])
                url_list.pop(i)
                # a = (time.time() - start_time) * len(url_list)
                # print('SUCCESS!!')
                # print(f'Currently on url number: {i}')
                # print(f'time taken: {a/len(url_list)}')
                if i > 0:
                    i -= randint(0, 1)
                else:
                    i += 0
            else:
                # print('No results')
                # print(f'Currently on url number: {i}')
                i += randint(1, 5)
        except:
            # print("Skipping. Connnection error")
            proxies.remove(proxy)
            # print(f'proxies left: {len(proxies)}')
            # print(f'total left: {len(url_list)}')
            return url_list, main_df, proxies
    return url_list, main_df, proxies

In [ ]:
url = 'https://www.zillow.com/homedetails/895-Yakima-Dr-Fremont-CA-94539/25028455_zpid/?fullpage=true'

In [ ]:
header = random.sample(ua, 1)[0]
soup = session_creator(ua, url)

In [ ]:
a = soup.findAll('div', {'class': 'fact-container'})
b = soup.findAll('div', {'class': 'fact-value'})

In [ ]:
z = []
y = []

In [ ]:
for num in a:
    z.append(num.text)
for num in b:
    y.append(num.text)

In [ ]:
z = [num.replace('  ','').replace('   ','').replace('\r','').replace('\n','').replace('\xa0','') for num in z]

In [ ]:
l = [re.findall(r'(.+)\:', num) for num in z]
# print(l)
k = [re.findall(r'\:(.+)', num) for num in z]
# print(k)
nd = {}
i = 0
while i < len(l):
    if len(l[i])>0:
        if l[i][0] in nd.keys():
            nd[l[i][0]].extend(k[i])
        else:
            nd[l[i][0]] = k[i]
        i+=1
    else:
        i+=1
for k,v in nd.items():
    nd[k] = ', '.join([num for num in v])
print(nd)

In [ ]:
y = [num.replace('  ','').replace('   ','').replace('\r','').replace('\n','').replace('\xa0','') for num in y]

In [ ]:
dict(zip(z,y))